In [1]:
import findspark
findspark.init()

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
.appName("Multi_Lineer_Regression") \
.master("local[4]") \
.config("spark.driver.memory","4g") \
.config("spark.executor.memory","4g") \
.getOrCreate()

22/10/22 10:28:58 WARN Utils: Your hostname, batuhan-VirtualBox resolves to a loopback address: 127.0.1.1; using 10.0.2.15 instead (on interface enp0s3)
22/10/22 10:28:59 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/10/22 10:29:00 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


# Veri Setini Okuma

In [3]:
df = spark.read.format("csv") \
.option("header", True) \
.option("inferSchema", True) \
.option("sep",",") \
.load("/home/batuhan/Desktop/datalar/Advertising16.csv")

In [4]:
df.toPandas().head(5)

22/10/22 10:29:19 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , TV, Radio, Newspaper, Sales
 Schema: _c0, TV, Radio, Newspaper, Sales
Expected: _c0 but found: 
CSV file: file:///home/batuhan/Desktop/datalar/Advertising16.csv


,_c0,TV,Radio,Newspaper,Sales
0,1,230.1,37.8,69.2,22.1
1,2,44.5,39.3,45.1,10.4
2,3,17.2,45.9,69.3,9.3
3,4,151.5,41.3,58.5,18.5
4,5,180.8,10.8,58.4,12.9


In [5]:
df2 = spark.read \
.option("header", True) \
.option("inferSchema", True) \
.option("sep",",") \
.csv("/home/batuhan/Desktop/datalar/Advertising16.csv")

In [6]:
df2.toPandas().head(5)

22/10/22 10:29:21 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , TV, Radio, Newspaper, Sales
 Schema: _c0, TV, Radio, Newspaper, Sales
Expected: _c0 but found: 
CSV file: file:///home/batuhan/Desktop/datalar/Advertising16.csv


,_c0,TV,Radio,Newspaper,Sales
0,1,230.1,37.8,69.2,22.1
1,2,44.5,39.3,45.1,10.4
2,3,17.2,45.9,69.3,9.3
3,4,151.5,41.3,58.5,18.5
4,5,180.8,10.8,58.4,12.9


Sütun İsimlerini Değiştirme

In [7]:
yeni_nitelikler = ["ıd","TV","Radio","Newspaper","Sales"]

In [8]:
df2 = df.selectExpr("_c0 as ıd","TV","Radio","Newspaper","Sales as label")

In [9]:
df2.toPandas().head(3)

22/10/22 10:29:22 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , TV, Radio, Newspaper, Sales
 Schema: _c0, TV, Radio, Newspaper, Sales
Expected: _c0 but found: 
CSV file: file:///home/batuhan/Desktop/datalar/Advertising16.csv


,ıd,TV,Radio,Newspaper,label
0,1,230.1,37.8,69.2,22.1
1,2,44.5,39.3,45.1,10.4
2,3,17.2,45.9,69.3,9.3


In [10]:
numerik_nitelikler = ["TV","Radio"]
label = ["label"]

In [11]:
df2.describe().toPandas().head()

22/10/22 10:29:23 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , TV, Radio, Newspaper, Sales
 Schema: _c0, TV, Radio, Newspaper, Sales
Expected: _c0 but found: 
CSV file: file:///home/batuhan/Desktop/datalar/Advertising16.csv


,summary,ıd,TV,Radio,Newspaper,label
0,count,200,200,200,200,200
1,mean,100.5,147.0425,23.264000000000024,30.553999999999995,14.022500000000003
2,stddev,57.879184513951124,85.85423631490805,14.846809176168728,21.77862083852283,5.217456565710477
3,min,1,0.7,0.0,0.3,1.6
4,max,200,296.4,49.6,114.0,27.0


In [12]:
df2.printSchema

<bound method DataFrame.printSchema of DataFrame[ıd: int, TV: double, Radio: double, Newspaper: double, label: double]>

# Veri Hazırlığı

In [13]:
from pyspark.ml.feature import VectorAssembler
vector_assembler = VectorAssembler().setInputCols(numerik_nitelikler).setOutputCol("features")

In [14]:
df2 = vector_assembler.transform(df2)

In [15]:
df2.toPandas().head(3)

22/10/22 10:29:29 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , TV, Radio, Newspaper, Sales
 Schema: _c0, TV, Radio, Newspaper, Sales
Expected: _c0 but found: 
CSV file: file:///home/batuhan/Desktop/datalar/Advertising16.csv


,ıd,TV,Radio,Newspaper,label,features
0,1,230.1,37.8,69.2,22.1,"[230.1, 37.8]"
1,2,44.5,39.3,45.1,10.4,"[44.5, 39.3]"
2,3,17.2,45.9,69.3,9.3,"[17.2, 45.9]"


## Regresyon Modeli

In [16]:
from pyspark.ml.regression import LinearRegression

In [17]:
lineer_params = (0.8)

In [18]:
lr_obj = LinearRegression(featuresCol = "features", labelCol = "label", regParam = lineer_params)

In [19]:
train_df, test_df = df2.randomSplit([0.8, 0.2], seed = 142)

In [20]:
model = lr_obj.fit(train_df)

22/10/22 10:29:34 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , TV, Radio, Newspaper, Sales
 Schema: _c0, TV, Radio, Newspaper, Sales
Expected: _c0 but found: 
CSV file: file:///home/batuhan/Desktop/datalar/Advertising16.csv


22/10/22 10:29:34 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
22/10/22 10:29:34 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.ForeignLinkerBLAS


22/10/22 10:29:35 WARN InstanceBuilder$NativeLAPACK: Failed to load implementation from:dev.ludovic.netlib.lapack.JNILAPACK
22/10/22 10:29:36 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , TV, Radio, Newspaper, Sales
 Schema: _c0, TV, Radio, Newspaper, Sales
Expected: _c0 but found: 
CSV file: file:///home/batuhan/Desktop/datalar/Advertising16.csv


# Model Testi

In [21]:
result_df = model.transform(test_df)

In [22]:
result_df.toPandas().head(5)

22/10/22 10:29:37 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , TV, Radio, Newspaper, Sales
 Schema: _c0, TV, Radio, Newspaper, Sales
Expected: _c0 but found: 
CSV file: file:///home/batuhan/Desktop/datalar/Advertising16.csv


,ıd,TV,Radio,Newspaper,label,features,prediction
0,4,151.5,41.3,58.5,18.5,"[151.5, 41.3]",17.191993
1,9,8.6,2.1,1.0,4.8,"[8.6, 2.1]",5.084467
2,11,66.1,5.8,24.2,8.6,"[66.1, 5.8]",7.972960
3,15,204.1,32.9,46.0,19.0,"[204.1, 32.9]",17.898416
4,25,62.3,12.6,18.3,9.7,"[62.3, 12.6]",8.939300


In [23]:
model.coefficients

DenseVector([0.0397, 0.1643])

In [24]:
model.intercept

4.398381341561637

In [25]:
model.summary.r2

0.8907063579921355

In [26]:
model.summary.rootMeanSquaredError

1.6941504975708543

In [27]:
model.summary.pValues

[0.0, 0.0, 0.0]

In [28]:
model.summary.tValues

[27.013031529232286, 19.444254956600705, 13.731187240961672]

# Model Seçimi

p Valuesler şu şekilde 
    [0.0,   0.0,     0.51710...., 3.81028......]
     TV     Radio     Newspaper    Label

Bu sebepten dolayı şu yöntemi izleyecek;
Geriye doğru eleme yönteminde Newspaper p değeri 0.05 değerinden büyük olduğu (0.5717) için modelden çıkarılır. 
Bunun için nümerik nitelikler listesinden 
Newspaper'ı çıkarmamız yeterli. Daha sonra o paragraftan itibaren tekrar çalıştıralım.

y = 2.935593134859488 + (0.0442 * TV) + (0.1978 * Radio)

In [29]:
import pandas as pd
d = {"TV":[100.0], "Radio":[10.0]}
pd_df = pd.DataFrame(data=d)
pd_df.head()

,TV,Radio
0,100.0,10.0


# AdvertisinDF'i PandasDF'ye çevirme

In [30]:
pd_df2 = df.toPandas()

22/10/22 10:29:40 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , TV, Radio, Newspaper, Sales
 Schema: _c0, TV, Radio, Newspaper, Sales
Expected: _c0 but found: 
CSV file: file:///home/batuhan/Desktop/datalar/Advertising16.csv


In [31]:
pd_df2.head(5)

,_c0,TV,Radio,Newspaper,Sales
0,1,230.1,37.8,69.2,22.1
1,2,44.5,39.3,45.1,10.4
2,3,17.2,45.9,69.3,9.3
3,4,151.5,41.3,58.5,18.5
4,5,180.8,10.8,58.4,12.9


# Belirli bir df seçme

In [32]:
pd_df2 = df.select(df["Sales"])

In [33]:
pd_df2 = pd_df2.toPandas()


In [34]:
pd_df2.head(5)

,Sales
0,22.1
1,10.4
2,9.3
3,18.5
4,12.9


In [35]:
pd_df

,TV,Radio
0,100.0,10.0


# PandasDataFrame Birleştirme İlk Raw'a Göre

In [36]:
new = pd_df2.iloc[[0]]

In [37]:
new

,Sales
0,22.1


In [38]:
pd_df4 = pd.concat([pd_df,new], axis = 1)

In [39]:
pd_df4

,TV,Radio,Sales
0,100.0,10.0,22.1


# Vector Assembler

In [40]:

assembler_vector = VectorAssembler(inputCols=["TV","Radio"], 
                                   outputCol="features")

# ÖENMLİ NOT ;
# VectorAssembler yapmak için PandasDF değil SparkDF 
# Olması Lazım bu sebepten pandas df'i sparkdf'e çevirdk(202)de

In [41]:
from pyspark.sql import SQLContext
sqlContext = SQLContext(spark)
spark_df = sqlContext.createDataFrame(pd_df4)

/home/batuhan/spark/python/pyspark/sql/context.py:112: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(
/home/batuhan/spark/python/pyspark/sql/pandas/conversion.py:474: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():
/home/batuhan/spark/python/pyspark/sql/pandas/conversion.py:486: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():


In [42]:
new_pd_df4 = assembler_vector.transform(spark_df)

In [43]:
new_pd_df4.toPandas().head(1)

,TV,Radio,Sales,features
0,100.0,10.0,22.1,"[100.0, 10.0]"


In [44]:
from pyspark.ml.regression import LinearRegression
lineer_reg = LinearRegression(featuresCol="features", labelCol="Sales",regParam = 1)

In [45]:
train_df, test_df = new_pd_df4.randomSplit([0.8,0.2],seed=42)

In [46]:
model = lineer_reg.fit(train_df)

22/10/22 10:29:46 WARN Instrumentation: [3edcac68] The standard deviation of the label is zero, so the coefficients will be zeros and the intercept will be the mean of the label; as a result, training is not needed.


In [48]:
# 100.000 TL TV HARCAMASI VE 10.000 TL RADİO HARCAMASI YAPRSAK TAHMİNİ SATIŞIMIZ NE OLUR

In [49]:
import pandas as pd
d2 = {"TV":[100.0], "Radio":[10.0]}
pd_df31 = pd.DataFrame(data=d)
pd_df31.head()

,TV,Radio
0,100.0,10.0


In [50]:
predict_df31 = spark.createDataFrame(pd_df31)
predict_df31.show()

/home/batuhan/spark/python/pyspark/sql/pandas/conversion.py:474: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():
/home/batuhan/spark/python/pyspark/sql/pandas/conversion.py:486: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():


+-----+-----+
|   TV|Radio|
+-----+-----+
|100.0| 10.0|
+-----+-----+



In [51]:
predict_df_vec31 = vector_assembler.transform(predict_df31)

In [52]:
model.transform(predict_df_vec31).show()

+-----+-----+------------+----------+
|   TV|Radio|    features|prediction|
+-----+-----+------------+----------+
|100.0| 10.0|[100.0,10.0]|      22.1|
+-----+-----+------------+----------+

